<h1 align=center><font size = 5>Web Scraping weather data</font></h1>

This script pulls a decade of weather data from http://www.estesparkweather.net/archive_reports.php and nicely cleans up and collects it into a dataframe


Lets import all required libraries

In [4]:
import pickle
import bs4
from bs4 import BeautifulSoup
import csv
import requests
import time
import pandas as pd
import urllib
import re
import pickle
from datetime import datetime
from tqdm import tqdm

Define the range of dates to pull data from through dynamic URL

In [5]:
range_date = pd.date_range(start = '1/1/2009',end = '11/01/2018',freq = 'M')
dates = [str(i)[:4] + str(i)[5:7] for i in range_date]
df_list = []
index = []

In [6]:
for k in tqdm(range(len(dates))):
    
		url = "http://www.estesparkweather.net/archive_reports.php?date="+ dates[k]
		#Our loop will run for the number of elements in dates
		page = requests.get(url)
		soup = BeautifulSoup(page.content, 'html.parser')
		table = soup.find_all('table')

		parsed_data = [row.text.splitlines() for row in table]
		parsed_data = parsed_data[:-9] 

		for l in range(len(parsed_data)):
		    parsed_data[l] = parsed_data[l][2:len(parsed_data[l]):3]
		    
		for i in range(len(parsed_data)):
		    c = ['.'.join(re.findall("\d+",str(parsed_data[i][j].split()[:5]))) for j in range(len(parsed_data[i]))]
		    df_list.append(c)
		    index.append(dates[k] + c[0])
		    f_index = [index[i] for i in range(len(index)) if len(index[i]) > 6]
		    data = [df_list[i][1:] for i in range(len(df_list)) if len(df_list[i][1:]) == 19]
		    final_index = [datetime.strptime(str(f_index[i]), '%Y%m%d').strftime('%Y-%m-%d') for i in range(len(f_index))]


100%|██████████| 118/118 [04:01<00:00,  2.04s/it]


Collect into a panda dataframe  

In [7]:
col = 		['Average temperature (°F)', 'Average humidity (%)',
            'Average dewpoint (°F)', 'Average barometer (in)',
            'Average windspeed (mph)', 'Average gustspeed (mph)',
            'Average direction (°deg)', 'Rainfall for month (in)',
            'Rainfall for year (in)', 'Maximum rain per minute',
            'Maximum temperature (°F)', 'Minimum temperature (°F)',
            'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
            'Minimum pressure', 'Maximum windspeed (mph)',
            'Maximum gust speed (mph)', 'Maximum heat index (°F)']
            
final_df = pd.DataFrame(data, columns = col, index = final_index)
final_df = final_df.astype(float)
display(final_df)


,Average temperature (°F),Average humidity (%),Average dewpoint (°F),Average barometer (in),Average windspeed (mph),Average gustspeed (mph),Average direction (°deg),Rainfall for month (in),Rainfall for year (in),Maximum rain per minute,Maximum temperature (°F),Minimum temperature (°F),Maximum humidity (%),Minimum humidity (%),Maximum pressure,Minimum pressure,Maximum windspeed (mph),Maximum gust speed (mph),Maximum heat index (°F)
2009-01-01,37.8,35.0,12.7,29.7,26.4,36.8,274.0,0.00,0.00,0.00,40.1,34.5,44.0,27.0,29.762,29.596,41.4,59.0,40.1
2009-01-02,43.2,32.0,14.7,29.5,12.8,18.0,240.0,0.00,0.00,0.00,52.8,37.5,43.0,16.0,29.669,29.268,35.7,51.0,52.8
2009-01-03,25.7,60.0,12.7,29.7,8.3,12.2,290.0,0.00,0.00,0.00,41.2,6.7,89.0,35.0,30.232,29.260,25.3,38.0,41.2
2009-01-04,9.3,67.0,0.1,30.4,2.9,4.5,47.0,0.00,0.00,0.00,19.4,0.0,79.0,35.0,30.566,30.227,12.7,20.0,32.0
2009-01-05,23.5,30.0,5.3,29.9,16.7,23.1,265.0,0.00,0.00,0.00,30.3,15.1,56.0,13.0,30.233,29.568,38.0,53.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-27,53.3,37.0,26.6,29.8,10.3,14.8,282.0,0.85,6.86,0.00,64.5,38.3,78.0,24.0,29.935,29.650,28.8,38.0,64.5
2018-10-28,49.4,48.0,25.2,29.9,5.3,7.6,282.0,0.85,6.86,0.00,67.3,30.9,90.0,19.0,30.040,29.663,18.4,25.3,67.3
2018-10-29,56.1,21.0,16.0,29.6,8.5,12.4,282.0,0.86,6.87,0.01,66.8,40.3,59.0,13.0,29.731,29.402,26.5,39.1,66.8
2018-10-30,31.5,87.0,28.2,29.9,1.9,3.1,275.0,0.99,7.00,0.01,40.3,27.8,92.0,60.0,30.121,29.623,9.2,16.1,40.2
